In [53]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
# @Time    : 2022/11/25 14:22
# @Author  : Wang Yujia
# @File    : data_extract_for_training.ipynb
# @Description : 为了training data分别提取csv并保存

# 1. Preparations
## 1.1 全局设置
1. output一共`data_key.shape[0]`个文件

In [42]:
# input data
GT_1_data_path = "../data/info_asymm/results/asc_symmetry/GT_asc_symmetry_P2_K=300.csv"
GT_2_data_path = "../data/SA_PT/results/PT_oneforall_P_K=300.csv"
# prod_embedding_output_path = "../data/prod_embedding_300.csv"
# prod_embedding_output_path = "../data/prod_embedding_100.csv"
prod_embedding_output_path = "../data/prod_embedding_60.csv"
target_data_key_path = "../data/target_datakey.csv"

# output path
train_root_path= "../data/train_60/"
train_file_head = "train_data_NP_"
train_file_tail= ".csv"
train_data_key_path = target_data_key_path

unique_features = ['id', 'bidincrement', 'bidfee','retail']
GT_features = ['bidincrement', 'bidfee','retail']
prod_embedding_features = ['id']

import pandas as pd
import numpy as np
from tqdm import tqdm

# 1.2 data读取与保存data_key

In [43]:
data_1 = pd.read_csv(GT_1_data_path, encoding="utf-8")
data_2 = pd.read_csv(GT_2_data_path, encoding="utf-8")
prod_embedding = pd.read_csv(prod_embedding_output_path,encoding="utf-8")
target_data_key = pd.read_csv(target_data_key_path,encoding="utf-8")

print(data_1.shape,data_2.shape,target_data_key.shape)
print(prod_embedding.head())

(605, 303) (605, 303) (1196, 4)
          0         1         2         3         4         5         6  \
0  0.121169  0.010344 -0.022827 -0.082939  0.014812  0.041113  0.026868   
1  0.059538  0.026735  0.015084  0.043650 -0.088015  0.064382 -0.052013   
2 -0.054701 -0.069931  0.115950  0.058021 -0.034754  0.145390 -0.024455   
3  0.002239  0.022239  0.000863  0.075391  0.008460 -0.055054 -0.178972   
4  0.057064  0.064496  0.016740  0.000108 -0.023041  0.008342 -0.098208   

          7         8         9  ...        52        53        54        55  \
0  0.032689 -0.049661 -0.052078  ...  0.050121 -0.008158  0.060036  0.039513   
1  0.057001  0.061996  0.100898  ...  0.012716 -0.037121  0.061844  0.043096   
2  0.004109  0.002671 -0.064163  ...  0.048623 -0.114022  0.166801 -0.068707   
3  0.150144  0.019524  0.042840  ... -0.047032  0.055298 -0.074437  0.035536   
4  0.018909  0.093087 -0.071372  ... -0.074892 -0.023282 -0.082430  0.038063   

         56        57        58     

In [44]:
data_1

,bidincrement,bidfee,retail,0,1,2,3,4,5,6,...,290,291,292,293,294,295,296,297,298,299
0,0.15,0.75,499.99,0.003988,0.003983,0.003978,0.003973,0.003969,0.003964,0.003959,...,0.002770,0.002766,0.002763,0.002759,0.002755,0.002752,0.002748,0.002745,0.002741,0.002737
1,0.15,0.75,169.99,0.005615,0.005595,0.005575,0.005556,0.005536,0.005516,0.005497,...,0.001716,0.001708,0.001700,0.001692,0.001684,0.001676,0.001668,0.001660,0.001652,0.001644
2,0.15,0.75,299.99,0.004491,0.004482,0.004473,0.004464,0.004455,0.004446,0.004437,...,0.002398,0.002392,0.002387,0.002381,0.002375,0.002370,0.002364,0.002359,0.002353,0.002348
3,0.15,0.75,89.99,0.008598,0.008541,0.008484,0.008427,0.008370,0.008314,0.008258,...,0.000603,0.000595,0.000587,0.000580,0.000572,0.000565,0.000557,0.000550,0.000543,0.000535
4,0.15,0.75,59.99,0.012513,0.012388,0.012263,0.012140,0.012017,0.011895,0.011775,...,0.000067,0.000064,0.000062,0.000060,0.000057,0.000055,0.000053,0.000051,0.000049,0.000047
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
600,0.12,0.60,359.99,0.004068,0.004062,0.004057,0.004051,0.004046,0.004041,0.004035,...,0.002708,0.002704,0.002700,0.002696,0.002692,0.002688,0.002684,0.002680,0.002676,0.002672
601,0.01,0.60,1339.00,0.003558,0.003556,0.003555,0.003553,0.003552,0.003550,0.003549,...,0.003131,0.003129,0.003128,0.003126,0.003125,0.003124,0.003122,0.003121,0.003120,0.003118
602,0.01,0.60,1719.00,0.003507,0.003506,0.003505,0.003504,0.003503,0.003501,0.003500,...,0.003175,0.003174,0.003173,0.003172,0.003170,0.003169,0.003168,0.003167,0.003166,0.003165
603,0.01,0.60,689.99,0.003778,0.003775,0.003772,0.003769,0.003765,0.003762,0.003759,...,0.002947,0.002944,0.002942,0.002939,0.002937,0.002934,0.002932,0.002929,0.002927,0.002924


# 2. 合并与拆分
## 2.1 合并大表
1. 合并的是target_key+GT的P vector
2. 合并之后会多一列`id`，后面会去掉这一列
3. 注意：**一定要让`target_data_key`成为左表**，这样连接之后，index可以一一对应

In [45]:
target_data_key.head()

,id,bidincrement,bidfee,retail
0,0,0.15,0.75,499.99
1,1,0.15,0.75,169.99
2,2,0.15,0.75,299.99
3,3,0.15,0.75,89.99
4,5,0.15,0.75,59.99


In [46]:
GT_1_withid= pd.merge(target_data_key,data_1,how="left",on=GT_features)
GT_1_withid,GT_1_withid.shape,np.any(pd.isna(GT_1_withid))

(        id  bidincrement  bidfee   retail         0         1         2  \
 0        0          0.15    0.75   499.99  0.003988  0.003983  0.003978   
 1        1          0.15    0.75   169.99  0.005615  0.005595  0.005575   
 2        2          0.15    0.75   299.99  0.004491  0.004482  0.004473   
 3        3          0.15    0.75    89.99  0.008598  0.008541  0.008484   
 4        5          0.15    0.75    59.99  0.012513  0.012388  0.012263   
 ...    ...           ...     ...      ...       ...       ...       ...   
 1191  1720          0.12    0.60   379.99  0.004025  0.004020  0.004015   
 1192  1721          0.12    0.60    46.99  0.012777  0.012646  0.012516   
 1193  1723          0.01    0.60  1719.00  0.003507  0.003506  0.003505   
 1194  1740          0.12    0.60   429.99  0.003938  0.003934  0.003930   
 1195  1748          0.01    0.60  1018.00  0.003631  0.003629  0.003627   
 
              3         4         5  ...       290       291       292  \
 0     0.003

In [47]:
GT_2_withid= pd.merge(target_data_key,data_2,how="left",on=GT_features)
GT_2_withid.head(),GT_2_withid.shape,np.any(pd.isna(GT_1_withid))

(   id  bidincrement  bidfee  retail         0         1         2         3  \
 0   0          0.15    0.75  499.99  0.001190  0.001197  0.001204  0.001212   
 1   1          0.15    0.75  169.99  0.004120  0.004129  0.004137  0.004145   
 2   2          0.15    0.75  299.99  0.001548  0.001557  0.001565  0.001574   
 3   3          0.15    0.75   89.99  0.014231  0.014097  0.013964  0.013831   
 4   5          0.15    0.75   59.99  0.026807  0.026188  0.025580  0.024982   
 
           4         5  ...           290           291           292  \
 0  0.001219  0.001227  ...  6.739960e-03  6.779261e-03  6.818779e-03   
 1  0.004153  0.004161  ...  1.316296e-03  1.302101e-03  1.288004e-03   
 2  0.001582  0.001591  ...  5.199096e-03  5.209410e-03  5.219623e-03   
 3  0.013697  0.013564  ...  4.994846e-05  4.879520e-05  4.766925e-05   
 4  0.024395  0.023818  ...  4.603279e-31  1.240681e-31  3.296644e-32   
 
             293           294           295           296           297  \
 0

## 2.2 拆分小表与保存


In [48]:
print(GT_1_withid.shape[0],GT_2_withid.shape[0],prod_embedding.shape[0])
print(GT_1_withid.shape[1],GT_2_withid.shape[1],prod_embedding.shape[1])
assert GT_1_withid.shape[0]==GT_2_withid.shape[0],"wrong!"

1196 1196 907
304 304 62


1. 把大表按照unique_features分成小表，
2. 小表里按照'id'，merge进去embedding的信息,然后删除'id'列，输出保存

In [49]:
df1 = pd.DataFrame([[1,2,3,4,5],[6,7,8,9,10]])
sum1 = np.sum(df1.iloc[0,1:1+2])
df1 = df1/sum1
df1, sum1

(     0    1    2    3    4
 0  0.2  0.4  0.6  0.8  1.0
 1  1.2  1.4  1.6  1.8  2.0,
 5)

In [50]:
LEN = 300
train_col = [str(i) for i in range(0,LEN)]

for i in tqdm(range(0,GT_1_withid.shape[0])):

    # 保存'id'
    id = GT_1_withid.loc[i,'id']
    # # 用LEN=300的data经过归一化变成LEN100
    # scale_sum_1 = np.sum(GT_1_withid.iloc[i,4:4+LEN])
    # scale_sum_2 = np.sum(GT_2_withid.iloc[i,4:4+LEN])
    # GT_1_withid_LEN = GT_1_withid.iloc[i,4:4+LEN]/scale_sum_1
    # GT_12_withid_LEN = GT_2_withid.iloc[i,4:4+LEN]/scale_sum_2
    #
    # train_tmp = pd.concat([pd.DataFrame(GT_1_withid_LEN).T,
    #                     pd.DataFrame(GT_1_withid_LEN).T],
    #                     ignore_index=True)

    train_tmp = pd.concat([pd.DataFrame(GT_1_withid.iloc[i,:]).T,
                            pd.DataFrame(GT_2_withid.iloc[i,:]).T],
                            ignore_index=True)
    # drop一些列，保持长度相等（LEN）
    train_tmp.drop(['id','bidincrement','bidfee','retail'],axis=1,inplace=True)

    # 按照'id'列找到embedding信息
    embedding = prod_embedding[prod_embedding['id'] == id].copy()
    # drop一些列，保持长度相等(LEN)
    embedding.drop(['id','desc'],axis=1,inplace=True)
    # 重命名，避免合并出问题
    train_tmp.columns = train_col
    # 合并
    train_pd = pd.concat([train_tmp,pd.DataFrame(embedding)], ignore_index=True)
    # save
    output_path = train_root_path+train_file_head+ str(i).zfill(4) + train_file_tail
    train_pd.to_csv(output_path,header=True,index=False,encoding="utf-8")

print("Done")

100%|██████████| 1196/1196 [00:03<00:00, 361.65it/s]

Done


## 2.3 拆分小表与保存【LEN<300】
1. 缩小training data的粒度
    - scale=3时相当于LEN=100
2. 是在原LEN=300的基础上进行的，必须读入GT长度为300的那2个表+长度为新粒度的embedding表

In [51]:
def rescale(data,SCALE):
    assert data.shape[1]==300,"!=300"
    data_new = pd.DataFrame()
    for i in range(0,data.shape[1],SCALE):
        if(i+SCALE<=data.shape[1]):  # 说明可以add SCALE个值
            tmp = np.sum(data.iloc[:,i:i+SCALE],axis=1)
        else:  # 加不够SCALE个值
            tmp = np.sum(data.iloc[:,i:],axis=1)
        # 拼接起来
        data_new = pd.concat([data_new,tmp],axis=1)
    assert data_new.shape[0] == data.shape[0],"Shape不等"

    return data_new

In [52]:
SCALE = 5
LEN = np.ceil(300/SCALE).astype(int)

print(f"SCALE={SCALE}, LEN={LEN}")

train_col = [str(i) for i in range(0,LEN)]

for i in tqdm(range(0,GT_1_withid.shape[0])):

    # 保存'id'
    id = GT_1_withid.loc[i,'id']
    # 先合并GT data
    train_tmp = pd.concat([pd.DataFrame(GT_1_withid.iloc[i,:]).T,
                            pd.DataFrame(GT_2_withid.iloc[i,:]).T],
                            ignore_index=True)

    # drop一些列，保持长度相等（LEN）
    train_tmp.drop(['id','bidincrement','bidfee','retail'],axis=1,inplace=True)
    # 执行Rescale
    scaled_data = rescale(train_tmp,SCALE)

    # 按照'id'列找到embedding信息
    embedding = prod_embedding[prod_embedding['id'] == id].copy()
    # drop一些列，保持长度相等(LEN)
    embedding.drop(['id','desc'],axis=1,inplace=True)
    # 重命名，避免合并出问题
    scaled_data.columns = train_col
    # 合并
    train_pd = pd.concat([scaled_data,pd.DataFrame(embedding)], ignore_index=True)
    # save
    output_path = train_root_path+train_file_head+ str(i).zfill(4) + train_file_tail
    train_pd.to_csv(output_path,header=True,index=False,encoding="utf-8")

print("Done")
print(train_root_path)

SCALE=5, LEN=60


100%|██████████| 1196/1196 [00:35<00:00, 33.58it/s]

Done
../data/train_60/
